## Content-based recommendation (using cosine similarity)
### Use article description to acquire similarity between articles

In [189]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import collaborative_filter as cf

In [190]:
#read Menswear data set
transaction_men_raw = pd.read_csv("../data/large_data/transaction_2019_567_Menswear.csv")
transaction_men_raw.head()

,Unnamed: 0,t_dat,customer_id,article_id,price,sales_channel_id,year,month,week,product_code,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,60,2019-05-01,00357b192b81fc83261a45be87f5f3d59112db7d117513...,743719001,0.050831,2,2019,5,18,743719,...,Shoes,F,Menswear,3,Menswear,27,Men Shoes,1020,Shoes,Cotton canvas trainers with a padded edge and ...
1,140,2019-05-01,0083ee250b3845008465de0e938d0ed2ae4f5bfde8b56e...,507431031,0.015237,2,2019,5,18,507431,...,Jersey inactive from s1,F,Menswear,3,Menswear,55,Contemporary Street,1005,Jersey Fancy,T-shirt in hard-washed slub cotton jersey with...
2,141,2019-05-01,0083ee250b3845008465de0e938d0ed2ae4f5bfde8b56e...,378447036,0.054220,2,2019,5,18,378447,...,Knitwear,F,Menswear,3,Menswear,23,Men Suits & Tailoring,1003,Knitwear,Jumper in fine-knit merino wool with a V-neck ...
3,142,2019-05-01,0083ee250b3845008465de0e938d0ed2ae4f5bfde8b56e...,657850001,0.030492,2,2019,5,18,657850,...,Knitwear,F,Menswear,3,Menswear,20,Contemporary Smart,1003,Knitwear,Jumper in a textured-knit wool blend with long...
4,143,2019-05-01,0083ee250b3845008465de0e938d0ed2ae4f5bfde8b56e...,598755015,0.013542,2,2019,5,18,598755,...,Light Basic Jersey,F,Menswear,3,Menswear,26,Men Underwear,1002,Jersey Basic,"Long, round-necked T-shirt in soft jersey with..."


In [5]:
print(transaction_men_raw.columns)

Index(['Unnamed: 0', 't_dat', 'customer_id', 'article_id', 'price',
       'sales_channel_id', 'year', 'month', 'week', 'product_code',
       'prod_name', 'product_type_no', 'product_type_name',
       'product_group_name', 'graphical_appearance_no',
       'graphical_appearance_name', 'colour_group_code', 'colour_group_name',
       'perceived_colour_value_id', 'perceived_colour_value_name',
       'perceived_colour_master_id', 'perceived_colour_master_name',
       'department_no', 'department_name', 'index_code', 'index_name',
       'index_group_no', 'index_group_name', 'section_no', 'section_name',
       'garment_group_no', 'garment_group_name', 'detail_desc'],
      dtype='object')


In [28]:
transaction_men_raw['details_description'] = transaction_men_raw['product_type_name'] + " " + \
                                             transaction_men_raw['product_group_name'] + " " + \
                                             transaction_men_raw['graphical_appearance_name'] + " " + \
                                             transaction_men_raw['colour_group_name'] + " " + \
                                             transaction_men_raw['perceived_colour_value_name'] + " " + \
                                             transaction_men_raw['perceived_colour_master_name'] + " " + \
                                             transaction_men_raw['department_name'] + " " + \
                                             transaction_men_raw['section_name'] + " " + \
                                             transaction_men_raw['garment_group_name'] + " " + \
                                             transaction_men_raw['detail_desc']
 

In [30]:
transaction_men_raw.details_description.value_counts()

T-shirt Garment Upper body Solid White Light White Light Basic Jersey Men Underwear Jersey Basic Round-necked T-shirt in soft cotton jersey.                                                                                                                                                                                                                                                                                                      1871
T-shirt Garment Upper body Front print White Light White Jersey Fancy Contemporary Smart Jersey Fancy T-shirt in printed cotton jersey.                                                                                                                                                                                                                                                                                                           1732
T-shirt Garment Upper body Solid Black Dark Black Light Basic Jersey Men Underwear Jersey Basic Round-necked T-shirt in so

In [5]:
content = transaction_men_raw.detail_desc.value_counts().index

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(content)
vectorizer.get_feature_names()

X.shape

(1923, 978)

In [6]:
similarity = cosine_similarity(X)

In [8]:
similarity.shape

(1923, 1923)

In [31]:
##subset data with article_id and detail_desc

article_desc_df = transaction_men_raw[['article_id','details_description']].drop_duplicates().dropna().reset_index(drop=True)
article_desc_df

,article_id,details_description
0,743719001,Sneakers Shoes Solid Grey Medium Dusty Grey Sh...
1,507431031,T-shirt Garment Upper body Treatment Black Dar...
2,378447036,Sweater Garment Upper body Melange Dark Blue D...
3,657850001,Sweater Garment Upper body Melange Dark Grey D...
4,598755015,T-shirt Garment Upper body Melange Greenish Kh...
...,...,...
5219,652346002,Underwear bottom Underwear Check Light Red Med...
5220,809223001,Trousers Garment Lower body Solid Greenish Kha...
5221,755780003,Sweater Garment Upper body Colour blocking Whi...
5222,786161002,Underwear bottom Underwear Solid White Light W...


In [32]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(article_desc_df['details_description'])
X.shape

(5224, 1081)

In [73]:
similarity = cosine_similarity(X)
similarity.shape

(5224, 5224)

In [74]:
similarity[0]

array([1.        , 0.08852791, 0.03685904, ..., 0.02976935, 0.0310341 ,
       0.07560741])

In [79]:
np.sort(similarity[0])

array([0.        , 0.00281989, 0.00288001, ..., 0.90912136, 0.92070483,
       1.        ])

In [37]:
#sort index of similarity 
sorted_sim_index = np.fliplr(np.argsort(similarity))

In [41]:
sorted_sim_index[1]

array([   1, 4450, 1994, ..., 3901, 1171, 3763], dtype=int64)

In [42]:
#the 10 most similar items to the item(index = 0)
articleID = article_desc_df.article_id[0]

similar_items_index = sorted_sim_index[0][1:11]

In [43]:
articleID

743719001

In [44]:
similar_items_index

array([2668, 2889, 5152,  701, 4957, 4979, 3582, 3635, 4188, 4009],
      dtype=int64)

In [48]:
article_desc_df.article_id[similar_items_index]

2668    601728002
2889    601728001
5152    601728027
701     601728013
4957    728836001
4979    766099001
3582    728788002
3635    671809009
4188    601728016
4009    728836003
Name: article_id, dtype: int64

In [191]:
mask_train = transaction_men_raw.week == 18
X_train = transaction_men_raw.loc[mask_train].reset_index(drop=True)
mask_test = transaction_men_raw.week == 19
X_test = transaction_men_raw.loc[mask_test].reset_index(drop=True)[['customer_id','article_id']]

In [196]:
Customer_IDs = X_train.customer_id.unique()
len(Customer_IDs)

5993

In [259]:
%%time

AOP_results = []
MAP_results = []
num_users = []
timeline = []

# week 18 - 31
for wk in range(18,31):
    mask_train = transaction_men_raw.week == wk
    X_train = transaction_men_raw.loc[mask_train].reset_index(drop=True)
    mask_test = transaction_men_raw.week == wk+1
    X_test = transaction_men_raw.loc[mask_test].reset_index(drop=True)[['customer_id','article_id']]

    #concatenate all text information
    X_train['details_description'] = X_train['product_type_name'] + " " + \
                                     X_train['product_group_name'] + " " + \
                                     X_train['graphical_appearance_name'] + " " + \
                                     X_train['colour_group_name'] + " " + \
                                     X_train['perceived_colour_value_name'] + " " + \
                                     X_train['perceived_colour_master_name'] + " " + \
                                     X_train['department_name'] + " " + \
                                     X_train['section_name'] + " " + \
                                     X_train['garment_group_name'] + " " + \
                                     X_train['detail_desc']

    #subset article_id and description column
    article_desc_df = X_train[['article_id','details_description']].drop_duplicates().dropna().reset_index(drop=True)

    #TF-IDF
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(article_desc_df['details_description'])

    #Calculate cosine similarity between articles
    similarity = cosine_similarity(X)

    #sort index of similarity
    sorted_sim_index = np.fliplr(np.argsort(similarity))

    #find customers in training and their purchases items
    Customer_IDs = X_train.customer_id.unique()

    #dict to save customer_id and recommended article_ids
    recommended_items = {}

    for customer in Customer_IDs:

        #find items purchased by customer in training
        purchased = list(X_train.loc[X_train.customer_id == customer].article_id)

        #empty list to store similarity array for each purchased item
        similarity_items = []

        #for each article_id, find its 12 most similar items' similarites
        for each in purchased:

            #article_id's index
            item_index = np.where(article_desc_df.article_id == each)[0]

            if item_index.size == 1:
                item_index = item_index[0]
                #use article_id's index to get its similarity vectors with others and append to list
                similarity_items.append(similarity[item_index])

        #if no similarity item, go to next customer
        if len(similarity_items) == 0:
            break
        
        #convert similarity_items list to np array
        similarity_items = np.asarray(similarity_items)

        #replace 1. with 0 in the similarity_items (1 is the item itself)
        similarity_items[similarity_items > 0.999] = 0

        #Get index of 12 highest similarity for each purchased item
        high_sim_index = np.argpartition(similarity_items, -12)[:,-12:].flatten()

        #Get similarity using above index info
        high_similarity = np.take(similarity_items, high_sim_index).flatten()

        #make a dataframe
        item_index_sim_df = pd.DataFrame({"item_idx":high_sim_index, "similarity":high_similarity})
        #sort by similarity and get the first 12 items
        item_index_sim_df = item_index_sim_df.sort_values('similarity', ascending=False).reset_index(drop=True)[0:12]

        #retrieve the article_id by index
        recommend_article_ids = list(article_desc_df.article_id[list(item_index_sim_df.item_idx)])

        #add user_id and recommended items to dict
        recommended_items[customer] = recommend_article_ids

    #calculate AOP@12 for each train-test group
    AOP12_CB, num_user = cf.AOP(recommended_items, X_test)
    AOP_results.append(AOP12_CB)

    MAP12_CB, num_user = cf.MAP(recommended_items, X_test)
    MAP_results.append(MAP12_CB)
    
    #append number of users
    num_users.append(num_user)

    #append week to timeline
    timeline.append('Wk_'+str(wk+1))

100%|████████████████████████████████████| 5993/5993 [00:00<00:00, 6666.29it/s]


Number of users: 379


100%|████████████████████████████████████| 5993/5993 [00:00<00:00, 6718.61it/s]


Number of users: 379


100%|████████████████████████████████████| 5098/5098 [00:00<00:00, 5664.44it/s]


...

Number of users: 84


100%|███████████████████████████████████| 8844/8844 [00:00<00:00, 11296.15it/s]


Number of users: 351


100%|███████████████████████████████████| 8844/8844 [00:00<00:00, 11457.10it/s]

Number of users: 351
Wall time: 5min 46s


In [260]:
print(AOP_results)
print(MAP_results)
print(num_users)

[0.028144239226033426, 0.02395209580838322, 0.017639902676399016, 0.018136335209505937, 0.022627965043695374, 0.01578282828282828, 0.019868496283590623, 0.01595214356929213, 0.0181732580037665, 0.01614434947768282, 0.017826086956521735, 0.010912698412698412, 0.018518518518518514]
[0.03853711762682739, 0.03128647106940519, 0.020841649901868882, 0.022698710503588557, 0.02959429208258797, 0.015353366821169851, 0.024033739749863253, 0.02145334678983732, 0.022276402714255815, 0.019049254928884558, 0.021501301838258362, 0.011377236079617032, 0.027437822808193194]
[379, 334, 411, 533, 534, 396, 583, 1003, 885, 702, 575, 84, 351]


In [262]:
#save results in a dataframe
result_df = pd.DataFrame({'timeline':timeline,
                          'AOP':AOP_results,
                          'MAP':MAP_results,
                          'num_users':num_users})

#save the dataframe in csv
#change file name for different period groups
result_df.to_csv("../data/MenswearResults/content-based.csv")

result_df.head()

,timeline,AOP,MAP,num_users
0,Wk_19,0.028144,0.038537,379
1,Wk_20,0.023952,0.031286,334
2,Wk_21,0.017640,0.020842,411
3,Wk_22,0.018136,0.022699,533
4,Wk_23,0.022628,0.029594,534


In [238]:
mask_train = transaction_men_raw.week == 20
X_train = transaction_men_raw.loc[mask_train].reset_index(drop=True)
mask_test = transaction_men_raw.week == 21
X_test = transaction_men_raw.loc[mask_test].reset_index(drop=True)[['customer_id','article_id']]


In [247]:
#concatenate all text information
X_train['details_description'] = X_train['product_type_name'] + " " + \
                                 X_train['product_group_name'] + " " + \
                                 X_train['graphical_appearance_name'] + " " + \
                                 X_train['colour_group_name'] + " " + \
                                 X_train['perceived_colour_value_name'] + " " + \
                                 X_train['perceived_colour_master_name'] + " " + \
                                 X_train['department_name'] + " " + \
                                 X_train['section_name'] + " " + \
                                 X_train['garment_group_name'] + " " + \
                                 X_train['detail_desc']

#subset article_id and description column
article_desc_df = X_train[['article_id','details_description']].drop_duplicates().dropna().reset_index(drop=True)

#TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(article_desc_df['details_description'])

#Calculate cosine similarity between articles
similarity = cosine_similarity(X)

#sort index of similarity
sorted_sim_index = np.fliplr(np.argsort(similarity))

#find customers in training and their purchases items
Customer_IDs = X_train.customer_id.unique()


In [255]:
pro_customer = '9058cc9742a41b8e1d1224fe8e8ea25190fd4e6125c1b4a0be2df07275c7f1e0'

In [256]:
#find items purchased by customer in training
purchased = list(X_train.loc[X_train.customer_id == pro_customer].article_id)
purchased

[706268015]

In [258]:
#dict to save customer_id and recommended article_ids
#recommended_items = {}

#for customer in Customer_IDs:

#find items purchased by customer in training
#purchased = list(X_train.loc[X_train.customer_id == pro_customer].article_id)

#empty list to store similarity array for each purchased item
similarity_items = []

#for each article_id, find its 12 most similar items' similarites
for each in purchased:

    #article_id's index
    item_index = np.where(article_desc_df.article_id == each)[0]

    if item_index.size == 1:
        item_index = item_index[0]
        #use article_id's index to get its similarity vectors with others and append to list
        similarity_items.append(similarity[item_index])

#convert sim_temp_list to np array
similarity_items = np.asarray(similarity_items)
similarity_items

array([[0.03733172, 0.03184119, 0.09414912, ..., 0.04126758, 0.05181664,
        0.03746181]])

In [249]:

#dict to save customer_id and recommended article_ids
recommended_items = {}

for customer in Customer_IDs:

    #find items purchased by customer in training
    purchased = list(X_train.loc[X_train.customer_id == customer].article_id)

    #empty list to store similarity array for each purchased item
    similarity_items = []

    #for each article_id, find its 12 most similar items' similarites
    for each in purchased:

        #article_id's index
        item_index = np.where(article_desc_df.article_id == each)[0]

        if item_index.size == 1:
            item_index = item_index[0]
            #use article_id's index to get its similarity vectors with others and append to list
            similarity_items.append(similarity[item_index])

    #convert sim_temp_list to np array
    similarity_items = np.asarray(similarity_items)

    #replace 1. with 0 in the similarity_items (1 is the item itself)
    similarity_items[similarity_items > 0.999] = 0

    print(customer)
    #Get index of 12 highest similarity for each purchased item
    high_sim_index = np.argpartition(similarity_items, -12)[:,-12:].flatten()

    #Get similarity using above index info
    high_similarity = np.take(similarity_items, high_sim_index).flatten()

    #make a dataframe
    item_index_sim_df = pd.DataFrame({"item_idx":high_sim_index, "similarity":high_similarity})
    #sort by similarity and get the first 12 items
    item_index_sim_df = item_index_sim_df.sort_values('similarity', ascending=False).reset_index(drop=True)[0:12]

    #retrieve the article_id by index
    recommend_article_ids = list(article_desc_df.article_id[list(item_index_sim_df.item_idx)])

    #add user_id and recommended items to dict
    recommended_items[customer] = recommend_article_ids



00cc7b20c3598962935bb62755b952a9c6517e0cb869195f352335fa998ac1da
01179a3b2c4f0a9d447b44a6d0e3bd6ba79ac85f1f40de787bc6efbcbd16b381
0198f1ec9513d4fab06a33dffba49746707b838f90ac4faed79986a0dee5c09f
022df8068a48bc81eb90cbb500c05d25e00fd4374948adf4146b0e71164f041c
0255d2ae6c3cb02f410df152c53942f1eb0d8e36d99a35e59581d108fbe7e48c
034ecb394573da6ddec92ddb08a402cea53370cce17a1836c3c7b2fabcd80368
037ec5bb01a1ea87eb873699dc40aac3b4e2bb2ba50a4c0a9b3594aec4a62d6c
03fa7262bf2b756b4e1c709b7cb12770759a60d3c17aa4a3e20cedbb237f2aaa
042a89d95d791e6b62e3eb7dc5d8109bae8fb12babdc3d981c780ca34334a739
047868ff416f0014c3fe35fc99115993c84cbefea12c9bd648dbb81c56a60d63
04af88d63d4d5a69c13278b0b0dc98925e2fc099c1a46443fa0b4baa1f981f42
05118b09ca00a6f66ad43e8cc25a6a476609d197edb265ee6b667f8844d9d899
06479355def71950c7197430f63082a3b86ae84438293ddd81d44fa1ca33bbfe
06b92dd450099becdb5572124300216c2854d21d1c8ac898c807526d5635cc43
07296a38c9756882d70661dbb9423543fcf19243620cb1c7bfcde0ebc4911335
078d3bd8741a0b37c05c68798

332a1bed41529c31c5b2e34de0c9fd70361dca6b6a615a80f7e5d2b405293b23
334e144cdb699b6118ce038cbe25f42174435eea433ae586667e0161b90aac2b
338b2a226055f4613cb48e793b1329c8235572e889af38f339fd1a67ec04ba1a
3401321752c829ef80d47753a49872ba353409fdc2b7d2ee9a060f306d349bb3
3451c42061c9e1386402865075c2b42902b07d70c8e82d9a43d1339731c3c663
345b0f9147215ef7e18d2117cb04eedf9c488177d530b43a43c5c86d35c63bcf
3467b238dc8d91f391f27e54725a52bbe04137097c3a3ea1ca3ec281b8dd5106
347aca487efe48b4020bfa9ea879d2ab3a88f332196170eefd5180b55aefd0ae
34878678f2b420f3010c314439d6ccdea581ceeeb06006d57f70a76dbdd13b61
3554fa2852c829611a2c8fc66f9eb047d73e588da7d433d331d271864d157f84
3639368aa6d65cece5d44d6a25b523045fd0838914084c02b3a1dae662aac3f7
36891c8207865bbcbc92b7da139c6318bec0856e807d9eb1570f4ab38f2f0a1d
36bec760c3d1790bf32db8d94e09729bdff14300e718a1cb27b87f2fe43d7d26
36ecd9f20fd80565a1604c510a3a20b7f70216b11e96c17d3aea38de6fa2131c
373d13886ccf026867a5c1c34cd4657525fcc27fef40b0718b3a80ccfa6913c1
38450461d9d8b685578f5393c

6136e8273f71a2732819afb3d0cf89f4dcd1f6d655a417ea4a70c0e65093c4ea
61387c9b71b2e2713906cc1af35fae169151f5c56e08bdafab7d53f76560addb
61c32e9385b9fe8ad847b1a9f60ec8c82947b161e792e99161c6251541370ef5
61deef5d0cdaca84e79118d1195ea97849cbb507fc8096ed1d7f1d7ad1eb9140
622af1b44d843bb68edc4e5094f233e76158214fc98bf9fdc91cb5bbd3c74508
62a45b29d0ed5e6dda358fb9b1c67259ff23da60d2fa0735c2f91d97ce47aa38
62eb7f6a247863fa029692beaeefdee2c64b1c34c7e865b981b5a9947e416198
63328c4a34c5c6dc7efd90df01252f5252af24bbd95958e8280cdaea4008e696
633c4a750daf2de331507e83c31f6ba9e50635bd4f4715d470822e9a03a8cf20
6365c47a0190be82ad9d120c29c163ec14d6059cddd843b3a5f2975f55641b2d
63cadd83ceb7df9be539d9a0e058d5a8dab31e2c1d69f2367e4fc94adb896f75
641a59aff760046e52cb8b55a16e05604bbdba94093968f26f18565aa14d5d38
6446cf40b9629a51d307f18676513a06bc15e37d4bd03ba4f9f13fa174c371d9
64ae1891f9595ce1ca0d94f29afd9f23a78f79feccae8e9f3ebeb22e66a3bcb4
64e01ae9b418bb0f937ec0ddb340f56a320a7e47dde0ed42bb7dfa31a70884ff
653b3736628c2faa09d745023

8d59e044e3c2a1cf754b7bacbc8939f09eca58497a0fc1d72928a51691f012fa
8dba102ad9065c5fb1f35041e60526ddb76b8dfd13048f4a112c85f39d843db3
8de939a2296299204ee5fff3db349d871aa76b82f1b405d368357de581e4d8b9
8e02609d10d12ddb67e8c3d711fe3e9eff09b12b9c71833b8106aa08e61c51f8
8e5d8d6d1e426d65be9d330127bb4120f0e1de0d356103d340e691ad44b9335c
8f18622ca85d4bd9847359e5d56a64e5efe78a9ae81993d0caab744009402f72
8f25da62b022276ad6ab05c0a30bfe5d57fd902cdad1276c9be64b3f50147356
90ef4c009693d3b4b36bad1f0cab9d7a665f6ce7a0cdc981b8dce4736090fa4e
90ff618e92164bcf7d731d009fa218110e8f5ba4d04d30d87ddab3d4f37fb3cb
9141f3865e04f24fe5e0b5d33045da2935661703597012c431ace267b11682ff
914af4497dc8f58b4efb14688e536b293b8d2421299ba5d45e9b83977d5dbbb8
92a87c176005f0fc615cda0d2bf811710b966e35bd1c47e53b9081d47b6d4d94
92e55649e0bd7cde18ced693590542a97081369f724f1fb57a7484ce7214c573
935cfe6d84df55a3f70556d10ec00de73b5f48b834bfc4ac7985647aabbf07af
93b1b4b7e05fb682b02a4358410eeafdc463b1289f43c2e05db0b8bbbe3e1805
93c2232f5a7d96fe10c001a87

d8ddef9b94d9b5c6a25689f1cce6e909426bfb5195ecbd29e39913d92234ca3e
d9c3c3daf3e1d041056669ee81706efe4bf3e6605a3554dcb01a996ee64f0712
d9c847d283f3107245fa2e9ddd6557859194400b50d1b43bd9f9cbc7d4dbdfca
d9e48ec55bea24da8b87a3ea85aebc91d64208e390eb9ee1005c6fbd82fa2d29
da3c828856c33c41742cf0378ea050f4c807d3c4a8169b206cd01f9410b8396a
dacfff48c6722163d11fb65bd5d6c0c8d19462823b376573830b6d27aa14660b
db6a2451928e49f08e867ec94c497f972954b3d46a9c921e0826400c08281f97
dc6b5d36254f1f9d922f1502a9c1bef528fe8c0a780bdfbdfc744e84c4cbaad6
dcb144330424a9001716e6a9b4914f26a929986893ce2c79cfb2e3b05628f7b8
dce84f1ff6ee22ae1a4683e0efe4293193fcf03c1f3a241a8e6dc0eb4c87fc7f
dd2ddeedc6fbe1834eac0e443205e234ac0edf9a182d8ccb8fb9432dafc0b081
ddd98228a86748c341fd9345a00d8f174774f7b0b05d7969fb65e3e76e9aaaef
de72b9f6f703e7becb656cb0457957d65f33521dd307238ab5b507d5ead1487f
ded0c41e1d330b3db5fe364cb29b88c8dd5a98772b66162d5c6166959d826ee5
df39fe72bf6379ceee1eda3cb8afc7cb60664ff4888799b1b6684a2ac9be2eea
df3dabc46912064e3d9ed5556

...

ecf178a7cbf0846defabd13a07b0f743069ec0ed30073822c64508a75305e1f6
ecff8bc224b1b42c4d2d8b20e8f342608d174f35ef7b984de3450534f43f23ba
ed16e0ca2df031154346238b69773b13e3a25ecc6789f69849c9806bfb84f413
ed1baa8624d6d4e0e2d573b3d42501651cc411a068d764fcad679a2c99612cab
ed941e33cd45b94d17c3fa5ab42ce9dd0781c7c02746aaf2fc1f4438b890cea5
ee29e3e2a0836458719b116ddf134bcf33f617c82b97c78719569c0c3f81acae
ee660cb7224fc636e8912ce5450367f9213393773be8c0308a1ff1d7a30892aa
ee6ac743ba4d67a824d299cdcd7e51bde75185ff4e67f3e9a820017249f09091
ee9263409dad15d2b82ba9c65c07eb543c1939aeee53a44fcde8809040299621
eeb334d9e463fa7ade7dbf5be330ad574b8eff589b70278e6902c6cbcee16baa
eed1df6dd50dd06f8334177f5cb3ce50ac37396dbf3c184bab83929ee0ed8bb4
eede0455f9b5cc7a3b2df49410f7fcfd0e27a0fa6af1cb6efe41b846249807c4
ef1946ad68f377f8ada288cd53aeb09f98548dbc670bba8f2ff6872331da34bb
ef2dadb46c5f6a172005ea5b6857279dba7be3eabc920a206d9ca33fb07eaa7f
ef968b2ba418e0b68ef6d9fe2dba03866f7ebf4b058dc4af468c9239c390356c
efba03a6351e4cd32ce24bdbf

0be644181ec2502de46d2d30a0a17daefae22b8ed418da1937d937f1266daefa
0c8eb152518f2ef165f71e25f8a64a0520cda84aebad2abbd5cad6ae1b03b800
0d4b309a32f4d5b3221dc66cf81b6103deef76a80be9096c7dac48242cd54971
0d7a38e8e41691f15d4135342efa73457b50a52043de853d4eb437765b285a17
0dc193de929fbd88d67b4f3c4a8f56b47a06c1ccf8c133e3a64b1c50359acd81
0de54ec1e8a0f20c63a97c9adf1dbcc82b6ca1d7376ea3499985e3d8dc549eb5
0eaf33b01f0bd3e50a2227ac13cb6df3dda271b6e59f127d7696089d2c58506b
0f8dd64476c5d7de6e5fb8a4ed9a484add9fda14492428d171251aae75b92780
0f9967004fe56b55ea0db8429e124d7bd763c882b6dcca6a2e72fcb4de41fb65
0fd882a895a738df4c0e10caeab812134a262401645a36abe2301b18bacf8350
0fea7acb6e8b7049d477025a396beb487ffea509ddc9ea2732911a08b3f721c4
10aa569a277985d02b62b1176deb680e3162909614bcfad060b8351a34419df7
10ccba7285244c9bbfef2c74d4634fdf9f70114091f772cbea4346b2378b5474
10e0e2d3b84e322cce7c53efd1a00abc8c49b233857c4a19ebd885f9cab9f3d6
1110ce5b4dc6836f520c7d75e8e458959a6536c560ab6f31e4e648b1daa40a15
112da9ffbc36ba83d60ea467a

37b819e8a7697d4714e9b1de9242e7b81d9288437a3c62c6f4b966c3f676a167
3859c7115701d968be49f4a4b1895607a8e4f15857d72c1a8e49da0a2a363c5c
388ca07c4496c2236561722c40c0b8d2bc31fbcdbffdbccb6aea0b5ade64a561
3894c1fc685457cb0fdc865510bf8317ae5651844535de59902c7c006b1a8ba6
38adb155408d333fef2809c821effcd0c47919b81ef0eab144e082bc06d8f834
38ce44dc98187e745c5ede63163e37db232c8c3c47ad6836f96fc0a6aee7893c
397b47e79d9a5429caf93d122c58f7a6aab3f89494ed04059a8fe1344bf80b6e
3ab61c27d7556bf36b023049e4bd1d8996964c92d2586c17bc32db9dcbd56d63
3b42838bff1898ad41566265b1c8f6aa3640617d726b307e2030e636f219f468
3bcab54ca90cc4c6b8fc46ee961f9f015d6a16250f8cd42c6bc8b6e1c21ee81b
3c1291f112afde1b13cee90b33f43cac2f1e9d23a0f9b9a7ad113a4f97058ee9
3cb6991699d539268edc6e94e0d5a54150fe62a8c51c5e94d2198dea35a509ba
3cd2d659125f30821d6c4b5a6ffcb6ea33127f32a477c9397f9dce08cadb1575
3d0b2ba032fcfad20a7cde3d183b821e2c5d4b3385ff80a3f80748b4b6428e58
3d0c7f09b7cc6986785fc98a6d1676b4a9769a085f649f3c15a0c3799fcde124
3d33f7bb7bd4dd4359dcc3f41

754acfa2e59051b105065ca60264976932bf40b19a45b35091a17b43b90782fe
75670a9bd4b2be9835b0caf086b0836358712dc468a0fa049e6217d43db5a87f
75892fd7b0f095656d42ce1cad253a754faf61d52982c4ce36c9b345eeae3ca5
758dda10c0e15be97d8046225cd12823cf8fc9f5dbfa01bdeb40ee0099269560
75986afcfa08019e413811b9441267b29703795d9f3d2d66163a51baf0774be0
75b3abe60c602826d1011ba9734ac96be9b9202262d59d75d620e5e471fe7736
7670eb5968995451ee81831f62b40ef484ce8ce3a864dcbe4471b40573af379e
7762a889fbf20797dacba8afdd3392467ef24c52af7e231731f6e71043acbaf1
783155c36450020fe175b6b21e3292669b4687390864c3cbe62a2e8f755eafd1
786789b047d78c89619410d357edc841c2abf06c52d3a5d83b3a966ff38e1a5a
7883c06d056a19659b97ab8452773696bd0d6d6368c0e31d20ac59ce66c60f72
78cb3a281b8634042b14accd9eba5a1fa747d96339bbfef76acf49abcd91ad4b
79c568bf65a32d438cd4d32e77b5ce3ea7e4e079f4c4adca21de798ebfd2d841
79c569cb3268955254abbfd760a6d299203cc9acf35030c40da8db26aa6a42fe
79e5ef2a811f90eabaa9d0a5c8208f685773b46f7c208de4cc927231d133bf1d
7a1cab6dddfbbbd3822143182

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [13]:
print(similarity.shape)
similarity

(2580, 2580)


array([[1.        , 0.08752494, 0.03553044, ..., 0.0414969 , 0.05856691,
        0.05257793],
       [0.08752494, 1.        , 0.10135107, ..., 0.31094455, 0.06615457,
        0.11935168],
       [0.03553044, 0.10135107, 1.        , ..., 0.07494133, 0.08330942,
        0.05427305],
       ...,
       [0.0414969 , 0.31094455, 0.07494133, ..., 1.        , 0.03937336,
        0.09324698],
       [0.05856691, 0.06615457, 0.08330942, ..., 0.03937336, 1.        ,
        0.58740676],
       [0.05257793, 0.11935168, 0.05427305, ..., 0.09324698, 0.58740676,
        1.        ]])

In [14]:
print(len(Customer_IDs))
Customer_IDs

5993


array(['00357b192b81fc83261a45be87f5f3d59112db7d117513c1e908e6a7021edc35',
       '0083ee250b3845008465de0e938d0ed2ae4f5bfde8b56ee9b59e6619d899e332',
       '00be0a263381af38132d31225e8fb12fbc527c654b446484fb672a78118f037d',
       ...,
       'ff666e1a64239927cc36e9b3f474ad6e613ff63742cb15c717ae51cc5e4c7ffd',
       'ffc21861854c3e3cf2dbef233cff8ba29a1dca5ff384651e9463a89674a2641c',
       'ffd70e82342cd4417db5458ae282c1450587734a9e00f81274afed82144f5540'],
      dtype=object)

In [166]:
# in a for loop
## for customer in Customer_IDs
customer = Customer_IDs[2]
customer

'00be0a263381af38132d31225e8fb12fbc527c654b446484fb672a78118f037d'

In [167]:
purchased = list(X_train.loc[X_train.customer_id == customer].article_id)
purchased

[589549018, 685604015, 732675001, 669385001]

In [168]:
#empty list to store similarity array for each purchased item
similarity_items = []

#for each article_id, find its 12 most similar items' similarites
for each in purchased:
    #article_id's index
    item_index = np.where(article_desc_df.article_id == each)[0][0]
    #use article_id's index to get its similarity vectors with others and append to list
    similarity_items.append(similarity[item_index])

similarity_items

[array([0.04261463, 0.20167606, 0.03974173, ..., 0.07246312, 0.0700987 ,
        0.11319982]),
 array([0.06674879, 0.03671266, 0.02434916, ..., 0.01362617, 0.2126426 ,
        0.09999439]),
 array([0.04233117, 0.20816835, 0.03580461, ..., 0.08029607, 0.03529698,
        0.0761465 ]),
 array([0.03460086, 0.18215487, 0.06643741, ..., 0.11445889, 0.090424  ,
        0.15733394])]

In [169]:
#convert sim_temp_list to np array
similarity_items = np.asarray(similarity_items)
similarity_items

array([[0.04261463, 0.20167606, 0.03974173, ..., 0.07246312, 0.0700987 ,
        0.11319982],
       [0.06674879, 0.03671266, 0.02434916, ..., 0.01362617, 0.2126426 ,
        0.09999439],
       [0.04233117, 0.20816835, 0.03580461, ..., 0.08029607, 0.03529698,
        0.0761465 ],
       [0.03460086, 0.18215487, 0.06643741, ..., 0.11445889, 0.090424  ,
        0.15733394]])

In [170]:
#replace 1. with 0 in the similarity_items (1 is the item itself)
similarity_items[similarity_items > 0.999] = 0
similarity_items

array([[0.04261463, 0.20167606, 0.03974173, ..., 0.07246312, 0.0700987 ,
        0.11319982],
       [0.06674879, 0.03671266, 0.02434916, ..., 0.01362617, 0.2126426 ,
        0.09999439],
       [0.04233117, 0.20816835, 0.03580461, ..., 0.08029607, 0.03529698,
        0.0761465 ],
       [0.03460086, 0.18215487, 0.06643741, ..., 0.11445889, 0.090424  ,
        0.15733394]])

In [171]:
#Get index of 12 highest similarity for each purchased item
high_sim_index = np.argpartition(similarity_items, -12)[:,-12:].flatten()

In [172]:
#Get similarity using above index info
high_similarity = np.take(similarity_items, high_sim_index).flatten()

In [173]:
#make a dataframe
item_index_sim_df = pd.DataFrame({"item_idx":high_sim_index, "similarity":high_similarity})
#sort by similarity and get the first 12 items
item_index_sim_df = item_index_sim_df.sort_values('similarity', ascending=False).reset_index(drop=True)[0:12]
item_index_sim_df

,item_idx,similarity
0,2012,0.975422
1,1421,0.949447
2,2268,0.947333
3,633,0.947333
4,634,0.915488
5,456,0.884409
6,264,0.825460
7,2464,0.806010
8,1967,0.798584
9,351,0.792591


In [185]:
list(item_index_sim_df.item_idx)

[2012, 1421, 2268, 633, 634, 456, 264, 2464, 1967, 351, 1963, 1207]

In [188]:
#retrieve the article_id by index
recommend_article_ids = list(article_desc_df.article_id[list(item_index_sim_df.item_idx)])
recommend_article_ids

[589549007,
 589549015,
 589549020,
 589549011,
 397688006,
 585355004,
 585355001,
 721063001,
 721063004,
 730013002,
 721063005,
 589556001]

In [125]:
np.argpartition(similarity_items, -12)[:,-12:]

array([[1207, 1963,  351, 1967, 2464, 2268, 2012,  634, 1421,  456,  264,
         633],
       [  86, 2060,  714,  310,  771,  326,  232, 1605,  905, 2290,  160,
        2568],
       [ 524, 1731, 1040, 1972,  806, 1282, 1429, 1094, 1677, 1430,  588,
        2315],
       [1448,  141, 1359,  842, 2249,  569, 1515,  941,   90, 2443, 2435,
        2266]], dtype=int64)

In [128]:
np.argpartition(similarity_items, -2)[:,-2:]

array([[1421, 2012],
       [ 160, 2568],
       [ 588, 2315],
       [ 569, 1515]], dtype=int64)

In [130]:
np.argpartition(similarity_items, -2)[:,-2:].flatten()

array([1421, 2012,  160, 2568,  588, 2315,  569, 1515], dtype=int64)

In [129]:
idx = np.argpartition(similarity_items, -2)[:,-2:]
np.take(similarity_items, idx)

array([[0.94944714, 0.97542231],
       [0.15764962, 0.1389232 ],
       [0.06126165, 0.17320611],
       [0.32754533, 0.06931327]])

In [131]:
np.take(similarity_items, idx).flatten()

array([0.94944714, 0.97542231, 0.15764962, 0.1389232 , 0.06126165,
       0.17320611, 0.32754533, 0.06931327])

In [132]:
dff = pd.DataFrame({'similarity':np.take(similarity_items, idx).flatten()},
                   index = np.argpartition(similarity_items, -2)[:,-2:].flatten())
dff

,similarity
1421,0.949447
2012,0.975422
160,0.157650
2568,0.138923
588,0.061262
2315,0.173206
569,0.327545
1515,0.069313


In [135]:
dff.sort_values('similarity', ascending=False)

,similarity
2012,0.975422
1421,0.949447
569,0.327545
2315,0.173206
160,0.157650
2568,0.138923
1515,0.069313
588,0.061262


In [ ]:


#concatenate all text information
X_train['details_description'] = X_train['product_type_name'] + " " + \
                                 X_train['product_group_name'] + " " + \
                                 X_train['graphical_appearance_name'] + " " + \
                                 X_train['colour_group_name'] + " " + \
                                 X_train['perceived_colour_value_name'] + " " + \
                                 X_train['perceived_colour_master_name'] + " " + \
                                 X_train['department_name'] + " " + \
                                 X_train['section_name'] + " " + \
                                 X_train['garment_group_name'] + " " + \
                                 X_train['detail_desc']

#subset article_id and description column
article_desc_df = X_train[['article_id','details_description']].drop_duplicates().dropna().reset_index(drop=True)

#TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(article_desc_df['details_description'])

#Calculate cosine similarity between articles
similarity = cosine_similarity(X)

#sort index of similarity
sorted_sim_index = np.fliplr(np.argsort(similarity))

#find customers in training and their purchases items
Customer_IDs = X_train.customer_id.unique()

#dict to store each 
recommend_items = {}

for customer in Customer_IDs:
    #find purchased article_id
    purchased = X_train.loc[X_train.customer_id == customer].article_id
    
    #empty list to store similarity array for each purchased item
    sim_temp_list = []
    
    #for each article_id, find its 12 most similar items' similarites
    for each in purchased:
        #article_id's index
        item_index = np.where(article_desc_df.article_id == each)[0][0]
        #use article_id's index to get its similarity vectors with others and append to list
        sim_temp_list.append(similarity[item_index])
    
    
        